Requires `TX_input_data_with_masses.csv` which is produced by the `compute_masses_TX.ipynb` notebook, or available on Box at `data/model inputs and outputs`

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import weibull_min

from celavi.simple_model import Context

np.random.seed(123)

In [6]:
scenario = 'bau'

## Define cost parameters

In [7]:
bau_cost_params = {'landfilling_transpo': 21.50,
                   'recycling_transpo': 16.0,
                   'coarse_grinding_cost_init': 55.13,
                   'fine_grinding_cost_init': 165.38,
                   'recycling_learning_rate': 0.05,
                   'recycling_strategic_value': 0.0,
                   'recycling_to_raw_material_revenue': -169.79,
                   'recycling_to_clinker_revenue': -10.37,
                   'recycling_landfilling_transpo': 0.15,
                   'recycling_to_clinker_transpo': 67.50,
                   'recycling_to_raw_material_transpo': 47.25}

mc_cost_params = {'landfilling_transpo': 21.50,
                   'recycling_transpo': 16.0,
                   'coarse_grinding_cost_init': 41.35,
                   'fine_grinding_cost_init': 143.33,
                   'recycling_learning_rate': 0.05,
                   'recycling_strategic_value': 10.0,
                   'recycling_to_raw_material_revenue': -191.02,
                   'recycling_to_clinker_revenue': -10.37,
                   'recycling_landfilling_transpo': 0.15,
                   'recycling_to_clinker_transpo': 67.50,
                   'recycling_to_raw_material_transpo': 47.25}

hc_cost_params = {'landfilling_transpo': 21.50,
                   'recycling_transpo': 16.0,
                   'coarse_grinding_cost_init': 27.56,
                   'fine_grinding_cost_init': 121.28,
                   'recycling_learning_rate': 0.05,
                   'recycling_strategic_value': 20.0,
                   'recycling_to_raw_material_revenue': -212.24,
                   'recycling_to_clinker_revenue': -10.37,
                   'recycling_landfilling_transpo': 0.15,
                   'recycling_to_clinker_transpo': 67.50,
                   'recycling_to_raw_material_transpo': 47.25}

## Import the USGS data

In [8]:
usgs = pd.read_csv("TX_input_data_with_masses.csv")

# drop any rows without years (should be none)
usgs.dropna(axis=0, subset=["year"], inplace=True)

# expand the dataset s.t. every turbine is one row
expand = pd.DataFrame(np.repeat(usgs.values,usgs['n_turbine'],axis=0))
expand.columns = usgs.columns
expand.drop(columns='n_turbine',inplace=True)

# remove raw input dataset
del usgs

# rename expanded dataset back to usgs
usgs = expand.copy()

print(len(usgs))
usgs.head()

22339


,year,t_cap_mw,ylat,xlong,blade_mass_tonnes,foundation_mass_tonnes
0,2001.0,1.051492,-100.376937,31.988169,13.006961,82.025352
1,2001.0,1.051492,-100.376937,31.988169,13.006961,82.025352
2,2001.0,1.051492,-100.376937,31.988169,13.006961,82.025352
3,2001.0,1.051492,-100.376937,31.988169,13.006961,82.025352
4,2001.0,1.051492,-100.376937,31.988169,13.006961,82.025352


## Run the DES model

Populate the components, run the DES model, extract the inventory levels

In [9]:
components = []
for _, turbine in usgs.iterrows():
    xlong = turbine["xlong"]
    ylat = turbine["ylat"]
    year = turbine["year"]
    blade_mass_tonnes = int(turbine["blade_mass_tonnes"])
    foundation_mass_tonnes = int(turbine['foundation_mass_tonnes'])
    
    components.append({
        "kind": "blade",
        "xlong": xlong,
        "ylat": ylat,
        "year": year,
        "mass_tonnes": blade_mass_tonnes,
    })
    
    components.append({
        "kind": "blade",
        "xlong": xlong,
        "ylat": ylat,
        "year": year,
        "mass_tonnes": blade_mass_tonnes,
    })
    
    components.append({
        "kind": "blade",
        "xlong": xlong,
        "ylat": ylat,
        "year": year,
        "mass_tonnes": blade_mass_tonnes,
    })
    
    components.append({
        "kind": "foundation",
        "xlong": xlong,
        "ylat": ylat,
        "year": year,
        "mass_tonnes": foundation_mass_tonnes,
    })
    
components = pd.DataFrame(components)

In [10]:
# All lifespans in the model are in discrete timesteps
timesteps_per_year = 4
start_yr=2000
end_yr=2050

lifespans = [20 * timesteps_per_year]
landfill_inventories = {}
landfill_mass_inventories = {}
K = 2.2   # DES Weibull shape parameter from Aubryn's study
min_lifespan = 10 * timesteps_per_year  # From Faulstitch et al

for lifespan in lifespans:
    L = lifespan  # DES Weibull scale parameter

    lifespan_fns = {
        "blade": lambda: weibull_min.rvs(K, loc=min_lifespan, scale=L-min_lifespan, size=1)[0],
#         "blade": lambda: weibull_min.rvs(K, loc=0, scale=L, size=1)[0],
        "foundation": lambda: 50,
    }

In [11]:
if scenario == 'bau':
    params=bau_cost_params
elif scenario == 'mc':
    params=mc_cost_params
elif scenario == 'hc':
    params=hc_cost_params
else:
    params=bau_cost_params

In [12]:
context = Context(min_year=start_yr, max_timesteps=timesteps_per_year*(end_yr-start_yr),
                  cost_params=params)
context.populate(components, lifespan_fns)
# calculate blade lifetimes in years based on the Weibull distribution
component_lifespans = [component.lifespan_timesteps/timesteps_per_year for component in context.components if component.kind == "blade"]

In [13]:
# run the DES model and get all material inventories
all_inventories = context.run()

In [14]:
all_inventories['landfill_component_inventory'].to_csv(scenario + '-landfill-component-inventory.csv')
all_inventories['landfill_mass_inventory'].to_csv(scenario + '-landfill-mass-inventory.csv')
all_inventories['virgin_component_inventory'].to_csv(scenario + '-virgin-component-inventory.csv')
all_inventories['virgin_material_inventory'].to_csv(scenario + '-virgin-mass-inventory.csv')
all_inventories['recycle_to_raw_component_inventory'].to_csv(scenario + '-recycle-raw-component-inventory.csv')
all_inventories['recycle_to_raw_mass_inventory'].to_csv(scenario + '-recycle-raw-mass-inventory.csv')
all_inventories['recycle_to_clinker_component_inventory'].to_csv(scenario + '-recycle-clinker-component-inventory.csv')
all_inventories['recycle_to_clinker_mass_inventory'].to_csv(scenario + '-recycle-clinker-mass-inventory.csv')
cost_hist_df = pd.DataFrame.from_dict(data=all_inventories['cost_history']).drop_duplicates()
cost_hist_df.to_csv(scenario + '-cost-history.csv')